In [1]:
from os.path import join, exists, isfile, isdir, abspath, dirname, basename, realpath
from os import makedirs, listdir, pardir, getcwd

from dataclasses import dataclass, field
from typing import Union
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import logging as log
from enum import Enum

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, Dropdown, Text, GridBox, VBox, HBox, Accordion, BoundedIntText, Checkbox, Layout, IntProgress, Tab, Image, Button
from IPython.display import display, Javascript

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#### Helper classes and functions

In [ ]:
#--------------------------------------------------
# Helper classes and enums
#--------------------------------------------------
@dataclass(frozen=False, order=False)
class CentralityAlgorithm:
    """
    A data-class to store centrality measures arguments and other options
    """
    name: str = field(default="")
    func: object = field(default=print)
    args: dict = field(default_factory=dict)
    options: dict = field(default_factory=dict)

@dataclass(frozen=False, order=False)
class CentralityAlgorithmOutput:
    """
    A data-class to store centrality measures results
    """
    name: str = field(default="")
    result: dict = field(default_factory=dict)
    options: dict = field(default_factory=dict)

class GRAPH_TYPE(str, Enum):
    BOTH = "both"
    DIRECTED = "directed"
    def __str__(self) -> str:
        return self.value

#--------------------------------------------------
# Helper functions
#--------------------------------------------------
